![QuantConnect Logo](https://cdn.quantconnect.com/web/i/qc_notebook_logo_rev0.png)
## Welcome to The QuantConnect Research Page
#### Refer to this page for documentation https://www.quantconnect.com/docs/research/overview#
#### Contribute to this template file https://github.com/QuantConnect/Lean/blob/master/Research/BasicQuantBookTemplate.ipynb

## QuantBook Basics

### Start QuantBook
- Add the references and imports
- Create a QuantBook instance

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Load in our startup script, required to set runtime for PythonNet
%run ./start.py

In [ ]:
class CustomDataType(PythonData):

    def GetSource(self, config: SubscriptionDataConfig, date: datetime, isLive: bool) -> SubscriptionDataSource:
        source = "https://www.dl.dropboxusercontent.com/s/d83xvd7mm9fzpk0/path_to_my_csv_data.csv?dl=0"
        return SubscriptionDataSource(source, SubscriptionTransportMedium.RemoteFile)

    def Reader(self, config: SubscriptionDataConfig, line: str, date: datetime, isLive: bool) -> BaseData:
        if not (line.strip()):
            return None

        data = line.split(',')
        obj_data = CustomDataType()
        obj_data.Symbol = config.Symbol

        try:
            obj_data.Time = datetime.strptime(data[0], '%Y-%m-%d %H:%M:%S') + timedelta(hours=20)
            obj_data["open"] = float(data[1])
            obj_data["high"] = float(data[2])
            obj_data["low"] = float(data[3])
            obj_data["close"] = float(data[4])
            obj_data.Value = obj_data["close"]

            # property for asserting the correct data is fetched
            obj_data["some_property"] = "some property value"
        except ValueError:
            return None

        return obj_data

    def __str__ (self):
        return f"Time: {self.Time}, Value: {self.Value}, SomeProperty: {self['some_property']}, Open: {self['open']}, High: {self['high']}, Low: {self['low']}, Close: {self['close']}"

In [ ]:
# Create an instance
qb = QuantBook()
symbol = qb.AddData(CustomDataType, "CustomDataType", Resolution.Hour).Symbol

startDate = datetime(2017, 8, 20)
endDate = startDate + timedelta(hours=48)
history = list(qb.History[CustomDataType](symbol, startDate, endDate, Resolution.Hour))

if len(history) == 0:
    raise Exception("No history data returned")